The goal of this notebook is to identify MP tools & publications IDs to use for the creation of new relations - POST /api/items-relations/{subjectId}/{objectId} - based on the relations identified here: https://lehkost.github.io/tools-dh-proceedings/index.html


In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

In [2]:
r = requests.get ('https://lehkost.github.io/tools-dh-proceedings/index.html')
input = r.text
soup=BeautifulSoup(input, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Tools mentioned in the proceedings of the annual ADHO conferences (2015–2019)
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
 </head>
 <body>
  <h3 style="text-align:center;">
   Laure Barbot ¹ · Frank Fischer ¹ ² · Yoann Moranville ¹ · Ivan Pozdniakov ²
  </h3>
  <p style="text-align:center;">
   <small>
    ¹ DARIAH-EU
    <br/>
    ² Higher School of Economics, Moscow
   </small>
  </p>
  <p style="text-align:center;">
   Berlin · Moscow · 6 December 2019
  </p>
  <p style="text-align:center;">
   *
  </p>
  <h2 style="text-align:center;">
   Tools mentioned in the proceedings of the annual ADHO conferences (2015–2019)
  </h2>
  <p style="text-align:center;">
   (
   <b>
    DH2015
   </b>
   , Sydney;
   <b>
    DH2016
   </b>
   , Kraków;
   <b>
    DH2017
   </b>
   , Montréal;
   <b>
    DH2018
   </b>
   , Ciudad de México;
   <b>
    DH2019
   </b>
   , Utrecht)
  </p>
  <p>
   This is a l

In [3]:
tapor_links = soup.find_all(href=re.compile("http://tapor.ca/tools/"))
for link in tapor_links:
    link['id']='tapor'

publi_titles = soup.find_all(href=re.compile("https://raw.githubusercontent.com/lehkost/ToolXtractor"))
for title in publi_titles:
    title['id']='publi'

In [4]:
mytable = pd.DataFrame()

ol_tag = soup.ol
for child in ol_tag.children:
    print(child)
    tool_name = child.find('b')
    if (not tool_name == -1):
        print(tool_name.contents[0])
        tapor_id = child.find('a')
        small_string=child.find('small')
        print(f'smallstring {small_string}')
        if ("no longer" in small_string.contents[0]):
            continue
        print(tapor_id.text)
        publis = child.find('ul')
        print(f'publications {publis}')
        list_of_publi = publis.find_all("li")
        tmp_list = []
        tmp_publi_list =[]
        for item in list_of_publi:
            print(f'single publi {item.a["href"]}')
            tmp_list.append(item.a["href"])
            
            if (item.a["href"]).endswith(".xml"):
                nextpage=requests.get((item.a["href"]))
                nextsoup = BeautifulSoup(nextpage.content, 'lxml')
                title=nextsoup.find('title')
                tmp_publi_list.append((title.text).replace('\n', ""))   
            
            elif (item.a["href"]).endswith(".txt"):
                r = requests.get((item.a["href"]))
                input = r.text
                regex = '^(.*?)\n'
                title = str(re.findall(regex, input))
                publi_title=str(title).strip("['']")
                tmp_publi_list.append(publi_title.replace('\n', ""))
            
            else:
                tmp_publi_list.append("wrong")

        tmp_data={'toolName':[tool_name.contents[0]], 'toolId':[tapor_id.text], 'toolPubs':[tmp_list], 'PubliName':[tmp_publi_list]}
        df_temp=pd.DataFrame(tmp_data)
        mytable=pd.concat([mytable,df_temp])



<li><p><b>Abbot</b> · <small>TAPoR:<a href="http://tapor.ca/tools/198" id="tapor">198</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/HOLMES_Martin_Whatever_Happened_to_Interchange_.xml" id="publi">xml/<b>2015</b>/HOLMES_Martin_Whatever_Happened_to_Interchange_.xml</a></li>
</ul></li>
Abbot
smallstring <small>TAPoR:<a href="http://tapor.ca/tools/198" id="tapor">198</a></small>
198
publications <ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/HOLMES_Martin_Whatever_Happened_to_Interchange_.xml" id="publi">xml/<b>2015</b>/HOLMES_Martin_Whatever_Happened_to_Interchange_.xml</a></li>
</ul>
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/HOLMES_Martin_Whatever_Happened_to_Interchange_.xml


<li><p><b>ABBYY FineReader</b> · <small>TAPoR:<a href="http://tapor.ca/tools/1389" id="tapor">1389</a></small></p><ul>
<li><a href="https://raw.git



<li><p><b>ANNIS</b> · <small>TAPoR:<a href="http://tapor.ca/tools/304" id="tapor">304</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2018/SCHROEDER_Caroline_T__Corpus_Linguistics_for_Multidisciplina.xml" id="publi">xml/<b>2018</b>/SCHROEDER_Caroline_T__Corpus_Linguistics_for_Multidisciplina.xml</a></li>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2019/WILLKOMM_Jens_The_CHQL_Query_Language_for_Conceptual_History.xml" id="publi">xml/<b>2019</b>/WILLKOMM_Jens_The_CHQL_Query_Language_for_Conceptual_History.xml</a></li>
</ul></li>
ANNIS
smallstring <small>TAPoR:<a href="http://tapor.ca/tools/304" id="tapor">304</a></small>
304
publications <ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2018/SCHROEDER_Caroline_T__Corpus_Linguistics_for_Multidisciplina.xml" id="publi">xml/<b>2018</b>/SCHROEDER_Caroline_T__Corpus_Linguistics_for_Multidisciplina.x

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2018/SCH_CH_Christof_Burrows__Zeta__Exploring_and_Evaluating_Vari.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2019/REBORA_Simone_Digital_Humanities_for_the_Study_of_Social_Rea.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2019/SARV_Mari_Digital_Folkloristics__Approaching_Variation_And_S.xml


<li><p><b>Anthologize</b> · <small>TAPoR:<a href="http://tapor.ca/tools/86" id="tapor">86</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/284.txt" id="publi">txt/<b>2017</b>/284.txt</a></li>
</ul></li>
Anthologize
smallstring <small>TAPoR:<a href="http://tapor.ca/tools/86" id="tapor">86</a></small>
86
publications <ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/284.txt" id="publi">txt/<b>2017</b>/284.txt</a></l

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2019/DOLLMAN_Melissa_Changing_Lanes__A_Reanimation_Of_Shell_Oil_s.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2019/ILVANIDOU_Maria_And_The_First_One_Now_Will_Later_Be_Last__Fo.xml


<li><p><b>Archive-It</b> · <small>TAPoR:<a href="http://tapor.ca/tools/1042" id="tapor">1042</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/99.xml" id="publi">xml/<b>2016</b>/99.xml</a></li>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/410.txt" id="publi">txt/<b>2017</b>/410.txt</a></li>
</ul></li>
Archive-It
smallstring <small>TAPoR:<a href="http://tapor.ca/tools/1042" id="tapor">1042</a></small>
1042
publications <ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/99.xml" id="publi">xml/<b>2016</b>/99.xml</a></li>
<li><

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2018/DALZIEL_Karin_Legacy_No_Longer__Designing_Sustainable_System.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2019/PAGE_Kevin_A_Layered_Digital_Library_for_Cataloguing_and_Res.xml


<li><p><b>Blender</b> · <small>TAPoR:<a href="http://tapor.ca/tools/807" id="tapor">807</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/DURIDANOV_Ludmil_Digital_Humanities_educational_vehicle.xml" id="publi">xml/<b>2015</b>/DURIDANOV_Ludmil_Digital_Humanities_educational_vehicle.xml</a></li>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2018/AFINOGUENOVA_Eugenia_V_The_Gallery_of_Queen_Isabel_in_the_Pr.xml" id="publi">xml/<b>2018</b>/AFINOGUENOVA_Eugenia_V_The_Gallery_of_Queen_Isabel_in_the_Pr.xml</a></li>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/

...

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/WITTEK_Stephen_DREaM__Distant_Reading_Early_Modernity.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/24.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/262.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/592.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/027.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/054.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/114.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/385.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/477.txt
single publi https://raw.githubusercontent.com/lehk

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/586.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/526.txt


<li><p><b>Wiki Map Project</b> · <small>no longer in TAPoR</small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/SIMPSON_John_Edward_Building_Better_Linked_Data___Ontol.xml" id="publi">xml/<b>2015</b>/SIMPSON_John_Edward_Building_Better_Linked_Data___Ontol.xml</a></li>
</ul></li>
Wiki Map Project
smallstring <small>no longer in TAPoR</small>


<li><p><b>WordCruncher</b> · <small>TAPoR:<a href="http://tapor.ca/tools/216" id="tapor">216</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2018/SCH_CH_Christof_Burrows__Zeta__Exploring_and_Evaluating_Vari.xml" id="publi">xml/<b>2018</b>/SCH_CH_Christof_Burrows__Zeta__Exploring_and_Evaluating_Vari.xml</a></li>
</ul></li>
WordCruncher
sm

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/FLANDERS_Julia_Challenges_of_an_XML_based_Open_Access_J.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/TAGLIAFERRI_Lisa_Mapping_the_Futures_of_Higher_Educatio.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/230.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/304.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/392.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/603.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/280.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/284.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/478.

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2019/MANDELL_Laura_C_Cutting_the_Gordian_Knot__Sustaining_Digital.xml


<li><p><b>YAGO</b> · <small>TAPoR:<a href="http://tapor.ca/tools/377" id="tapor">377</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/161.xml" id="publi">xml/<b>2016</b>/161.xml</a></li>
</ul></li>
YAGO
smallstring <small>TAPoR:<a href="http://tapor.ca/tools/377" id="tapor">377</a></small>
377
publications <ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/161.xml" id="publi">xml/<b>2016</b>/161.xml</a></li>
</ul>
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/161.xml


<li><p><b>Zenodo</b> · <small>TAPoR:<a href="http://tapor.ca/tools/596" id="tapor">596</a></small></p><ul>
<li><a href="https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2018/BETZ_Katrin_T

single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2015/BRADLEY_John_How_about_Tools_for_the_whole_range_of_sch.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/333.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/350.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/xml/2016/365.xml
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/232.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/385.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/455.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/548.txt
single publi https://raw.githubusercontent.com/lehkost/ToolXtractor/master/data/DH/txt/2017/583.txt
single publi https://raw.githubusercontent.com/l

In [5]:
mytable.head(10)

,toolName,toolId,toolPubs,PubliName
0,Abbot,198,[https://raw.githubusercontent.com/lehkost/Too...,[Whatever Happened to Interchange?]
0,ABBYY FineReader,1389,[https://raw.githubusercontent.com/lehkost/Too...,[eLexicon. Dictionary of Polish Medieval Latin...
0,Academia.edu,1144,[https://raw.githubusercontent.com/lehkost/Too...,[Humanities Commons: Collaboration and Collect...
0,Adobe After Effects,1217,[https://raw.githubusercontent.com/lehkost/Too...,[First We Feel Then We Fall - Multimedia Adapt...
0,Adobe Illustrator,1215,[https://raw.githubusercontent.com/lehkost/Too...,[New Approaches to Women’s Writing Virtual Res...
0,Adobe InDesign,1216,[https://raw.githubusercontent.com/lehkost/Too...,"[Translation, Annotation and Knowledge Modelli..."
0,Advene,940,[https://raw.githubusercontent.com/lehkost/Too...,[Prendre en Compte le Contexte d’Usage et le C...
0,Alveo,530,[https://raw.githubusercontent.com/lehkost/Too...,[Alveo: Software Engineering with Humanities R...
0,ANNIS,304,[https://raw.githubusercontent.com/lehkost/Too...,[Corpus Linguistics for Multidisciplinary Rese...
0,Annotation Studio,443,[https://raw.githubusercontent.com/lehkost/Too...,[Defining the Core Entities of an Environment ...


In [6]:
mytable.iloc[1].PubliName

['eLexicon. Dictionary of Polish Medieval Latin: from TEI encoding to an',
 'Improving OCR of Black Letter in Historical Newspapers: The Unreasonable Effectiveness of HTR Models on Low-Resolution Images']

In [7]:
mpdata = mpd()
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)

getting data from local repository...
getting data from local repository...


In [8]:
tapor_tools = df_tool_flat[(df_tool_flat['source.label']=='TAPoR')]
tapor_tools[['id','category','label','accessibleAt','persistentId','source.label','sourceItemId']]

,id,category,label,accessibleAt,persistentId,source.label,sourceItemId
0,50051,tool-or-service,140kit,[https://github.com/WebEcologyProject/140kit],SIU1nO,TAPoR,937
6,50052,tool-or-service,3DVIA Virtools,[],zrfCly,TAPoR,1229
7,50056,tool-or-service,4th Dimension,[http://www.4d.com/products/4d2004/4dstandarde...,rdwzoM,TAPoR,783
8,50057,tool-or-service,80legs,[http://80legs.com/],XsXzlp,TAPoR,648
9,50059,tool-or-service,960 Grid System,[https://960.gs/],uo4gCA,TAPoR,1454
...,...,...,...,...,...,...,...
1802,27963,tool-or-service,Zoomify,[],NR5Mbp,TAPoR,1234
1803,52421,tool-or-service,Zotero,[https://www.zotero.org/],dY3lIq,TAPoR,798
1804,52424,tool-or-service,ZotFile,[http://zotfile.com/],KlkkWa,TAPoR,945
1805,52427,tool-or-service,Zotpress,[https://wordpress.org/plugins/zotpress/],YoTO6a,TAPoR,1443


In [12]:
toolsMerged = mytable.join(tapor_tools.set_index('sourceItemId'), on=['toolId'], how="left",)
cleanTools = toolsMerged[['toolName','toolId','label', 'persistentId', 'toolPubs', 'PubliName']]

In [10]:
publications_list = df_publication_flat[['id','category','label','persistentId','accessibleAt','source.label','sourceItemId']]
publications_list.head(5)

,id,category,label,persistentId,accessibleAt,source.label,sourceItemId
0,42142,publication,1947Partition On The Margins - The Untold Test...,FuBh5c,[https://dh2017.adho.org/abstracts/081/081.pdf],dblp: DH Publications,conf/dihu/AroraM17
1,42143,publication,1 Million Dutch Newspaper Images available for...,EppSCv,[http://dh2016.adho.org/abstracts/193],dblp: DH Publications,conf/dihu/KleppeE16
2,10414,publication,3D-ICONS -- 3D Digitisation of Icons of Europe...,jOum8c,[http://3dicons-project.eu/guidelines-and-case...,SSK Zotero Resources,XY49BQNJ
3,42144,publication,3D Scanning for Preservation - Difficulties an...,A6IsdT,[http://dh2016.adho.org/abstracts/127],dblp: DH Publications,conf/dihu/FeredayMWS16
4,42145,publication,3rd International Workshop on Computational Hi...,oKWLnW,[http://ceur-ws.org/Vol-1632],dblp: DH Publications,conf/dihu/2016histoinfo


In [13]:
def clean_names(publi_title):
    cleanName = re.sub("[^A-Za-z0-9]+", "", publi_title,0,re.IGNORECASE)
    return cleanName

publications_list["clean_label"] = publications_list['label'].apply(clean_names)
everyPubli = cleanTools.explode('PubliName')
everyPubli["clean_label"] = everyPubli['PubliName'].apply(clean_names)

<ipython-input-13-a61b983d29ec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publications_list["clean_label"] = publications_list['label'].apply(clean_names)


In [16]:
fulldf = pd.merge(everyPubli, publications_list, on=['clean_label'], how="left")
cleandf = fulldf[['toolName','toolId','label_x', 'persistentId_x', 'toolPubs', 'PubliName', 'clean_label', 'persistentId_y']]
cleandf2=cleandf.dropna(subset=['persistentId_x', 'persistentId_y'])
cleandf2

,toolName,toolId,label_x,persistentId_x,toolPubs,PubliName,clean_label,persistentId_y
3,Academia.edu,1144,Academia.edu,y2dwph,[https://raw.githubusercontent.com/lehkost/Too...,Humanities Commons: Collaboration and Collecti...,HumanitiesCommonsCollaborationandCollectiveAct...,B0OnQw
4,Academia.edu,1144,Academia.edu,y2dwph,[https://raw.githubusercontent.com/lehkost/Too...,Humanities Commons: Collaboration and Collecti...,HumanitiesCommonsCollaborationandCollectiveAct...,PFz1ob
5,Adobe After Effects,1217,Adobe After Effects,U6hzqf,[https://raw.githubusercontent.com/lehkost/Too...,First We Feel Then We Fall - Multimedia Adapta...,FirstWeFeelThenWeFallMultimediaAdaptationofJoy...,zL6pX6
10,ANNIS,304,ANNIS (ANNotation of Information Structure),E8DhE2,[https://raw.githubusercontent.com/lehkost/Too...,Corpus Linguistics for Multidisciplinary Resea...,CorpusLinguisticsforMultidisciplinaryResearchC...,7sA8oL
12,Annotation Studio,443,Annotation Studio,VOENAw,[https://raw.githubusercontent.com/lehkost/Too...,Defining the Core Entities of an Environment f...,DefiningtheCoreEntitiesofanEnvironmentforTextu...,v7LLbL
...,...,...,...,...,...,...,...,...
1264,Zotero,798,Zotero,dY3lIq,[https://raw.githubusercontent.com/lehkost/Too...,Rebuilding Digital Harlem for Sustainability a...,RebuildingDigitalHarlemforSustainabilityandChange,VL67Pq
1265,Zotero,798,Zotero,dY3lIq,[https://raw.githubusercontent.com/lehkost/Too...,Creating An EpiDoc Corpus for Ancient Sicily,CreatingAnEpiDocCorpusforAncientSicily,awZ4jF
1267,Zotero,798,Zotero,dY3lIq,[https://raw.githubusercontent.com/lehkost/Too...,Embedding Digital Humanities in a Classics Mas...,EmbeddingDigitalHumanitiesinaClassicsMasterPro...,Dmwj4T
1269,Zotero,798,Zotero,dY3lIq,[https://raw.githubusercontent.com/lehkost/Too...,Archivlo: Digitizing the Archival Research Wor...,ArchivloDigitizingtheArchivalResearchWorkflow,at1uk7




POST /api/items-relations/{subjectId}/{objectId} 

https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/items-relations/YZhok6/yxvPh5

{
   "code": "is-mentioned-in"
   
}

